In [1]:
import os
import pickle
import torch
print(torch.__version__)
import numpy as np
from math import ceil
from model_vc import Generator
from model_bl import D_VECTOR
from collections import OrderedDict


import librosa
from synthesis import build_model
from synthesis import wavegen

from make_spect import makeSpect

import soundfile as sf
import warnings
warnings.filterwarnings('ignore')

def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad


1.2.0


# AutoVC

In [2]:
autovc_checkpoint = 'checkpoints/autovc_1000000.pt'
device = 'cuda:0'
G = Generator(32,256,512,32).eval().to(device)

g_checkpoint = torch.load(autovc_checkpoint, map_location=torch.device('cuda'))

#G.load_state_dict(g_checkpoint['model'])
G = g_checkpoint

###### AUDIO AND CHECKPOINT PATH   ---    GENERATE MEL

In [3]:


original_audio = "../vivos_only_wavs/VIVOSSPK22/VIVOSSPK22_001.wav" # first audio
ref_audio = "../vivos_only_wavs/VIVOSSPK21/VIVOSSPK21_001.wav" # second audio


mel_org = makeSpect(original_audio)
mel_ref = makeSpect(ref_audio)

In [4]:
x_org = mel_org
x_org, len_pad = pad_seq(x_org)
uttr_org = torch.FloatTensor(x_org[np.newaxis, :, :]).to(device)

org_idx, ref_idx = 22,21

emb1,emb2 = np.zeros(65,dtype="float32"),np.zeros(65,dtype="float32")
emb1[org_idx-1],emb2[ref_idx-1]=1,1

emb_org = torch.FloatTensor(emb1).unsqueeze(0).cuda()
emb_ref = torch.FloatTensor(emb2).unsqueeze(0).cuda()

In [5]:
with torch.no_grad():
    _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_ref)
    
if len_pad == 0:
    uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
else:
    uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()


#waveform = wavegen(model, c=uttr_trg)   
#sf.write('test.wav', waveform, 16000,subtype='PCM_24')

# Waveglow

In [1]:
import torch
import numpy as np
print(torch.__version__)
waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')

1.2.0


Using cache found in /home/trinhan/.cache/torch/hub/nvidia_DeepLearningExamples_torchhub


In [2]:
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

WaveGlow(
  (upsample): ConvTranspose1d(80, 80, kernel_size=(1024,), stride=(256,))
  (WN): ModuleList(
    (0): WN(
      (in_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
        (3): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
        (4): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
        (5): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
        (6): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
        (7): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
      )
      (res_skip_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
        (1): Conv1d(51

In [11]:
#mel = x_identic_psnt[0,:,:,:].permute(0,2,1)
mel = np.load("../database_sa2_Feb29_Mar06_Mar17_toApril06_cleaned_utt_0000153098-1.npy")
mel = torch.FloatTensor(mel).unsqueeze(0).permute(0,2,1).to("cuda")

with torch.no_grad():
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()
rate = 22050

In [12]:
from IPython.display import Audio
Audio(audio_numpy, rate=rate)

# Wavenet

In [49]:
#if len_pad == 0:
#    uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
#else:
#    uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()

device = torch.device("cuda")
model = build_model().to(device)
checkpoint = torch.load("../checkpoint_step001000000_ema.pth",map_location=torch.device('cuda'))
model.load_state_dict(checkpoint["state_dict"])

waveform = wavegen(model, c=uttr_trg)   
sf.write('test.wav', waveform, 16000,subtype='PCM_24')

100%|██████████| 57088/57088 [26:13<00:00, 36.27it/s]
